<a href="https://colab.research.google.com/github/mariaflbss/API-NEXUS/blob/main/filtragem_de_dados_dos_munic%C3%ADpios_de_S%C3%A3o_Paulo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importação das bibliotecas
from google.colab import drive
import pandas as pd

In [ ]:
#Mount no Gdrive do diretório onde estarão os arquivos e a base de dados
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#cd muda o diretório de trabalho para a pasta onde a base de dados se  encontra
#É necessário criar um atalho com a pasta do projeto no Drive para o diretório especificado durante o Mount
%cd "/content/drive/MyDrive/Colab Notebooks/API1S"

/content/drive/MyDrive/Colab Notebooks/API1S


In [ ]:
#declaração de variáveis usadas num looping para que não seja necessário ler os arquivos .csv um a um
#a cada iteração do while o nome do arquivo muda para o ano seguinte(de 2019 a 2014) e o .csv é adicionado a lista_arquivos
ano = 2019
lista_arquivos = []

while ano <= 2024:
  df = pd.read_csv(f"EXP_{ano}_MUN.csv", sep=";", encoding="latin1",on_bad_lines = "skip", usecols='''CO_MES SH4 CO_PAIS SG_UF_MUN CO_MUN KG_LIQUIDO VL_FOB'''.split())
  lista_arquivos.append(df)
  ano += 1

#leitura das tabelas de correspondência de códigos e nomes do município e código SH
#colunas renomeadas para ficarem iguais nos dois arquivos
df_mun = pd.read_csv("UF_MUN.csv", sep = ";", encoding="ISO-8859-1", on_bad_lines="skip", usecols='''CO_MUN_GEO NO_MUN_MIN SG_UF'''.split())
df_mun = df_mun.rename(columns= {"CO_MUN_GEO" : "CO_MUN", "SG_UF": "SG_UF_MUN"})

df_mun = df_mun[df_mun["SG_UF_MUN"] == "SP"]

df_sh4 = pd.read_csv("NCM_SH.csv", sep = ";", encoding="ISO-8859-1", on_bad_lines="skip", usecols='''CO_SH4 NO_SH4_POR'''.split())
df_sh4 = df_sh4.rename(columns= {"CO_SH4" : "SH4"})

#for percorre a lista de arquivos e cria uma nova coluna "VALOR AGREGADO" que recebe o resultado da conta para valor agregado
# > 0 evita que haja divisão por 0
#carga_num agrupa 3 colunas de acordo com o valor agregado e calcula a média
# == "SP" faz com que apenas o estado de SP seja analizado
#top5 sorteia os 5 maiores valores agregados de carga_num
for df in lista_arquivos:

  #Calcula o Valor agregado evitando divisão por zero
  df["VALOR_AGREGADO"] = df["VL_FOB"] / df["KG_LIQUIDO"]

  df = df[df["KG_LIQUIDO"] > 0]

  #converter o CO_MUN para string em ambos os dataframes
  df["CO_MUN"] = df["CO_MUN"].astype(str)
  df_mun["CO_MUN"] = df_mun["CO_MUN"].astype(str)

  df["SH4"] = df["SH4"].astype(str)
  df_sh4["SH4"] = df_sh4["SH4"].astype(str)

  #para substituir o código do município pelo nome corretamente
  df = df[df["SG_UF_MUN"] == "SP"]
  df = df.merge(df_mun, how="inner", on="CO_MUN")

  df = df.merge(df_sh4, how="inner", on="SH4")
  carga_mun = df.groupby(["NO_MUN_MIN", "SG_UF_MUN_x", "NO_SH4_POR"])['VALOR_AGREGADO'].mean().reset_index()

  top5 = carga_mun.sort_values(by="VALOR_AGREGADO", ascending=False).head(5)
  print(top5)


<ipython-input-5-d765204af71e>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["CO_MUN"] = df["CO_MUN"].astype(str)
<ipython-input-5-d765204af71e>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["SH4"] = df["SH4"].astype(str)


      NO_MUN_MIN SG_UF_MUN_x  \
8624     Itapevi          SP   
21141  São Paulo          SP   
10176     Jarinu          SP   
1922     Barueri          SP   
1728     Barueri          SP   

                                              NO_SH4_POR  VALOR_AGREGADO  
8624                              Tiocompostos orgânicos   278200.000000  
21141  Obras cartográficas de qualquer espécie, inclu...   175000.000000  
10176  Elementos químicos radioactivos e isótopos rad...    98865.000000  
1922   Âmbar-cinzento, castóreo, algália e almíscar; ...    50814.150000  
1728   Ouro (incluído o ouro platinado), em formas br...    49881.730769  


<ipython-input-5-d765204af71e>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["CO_MUN"] = df["CO_MUN"].astype(str)
<ipython-input-5-d765204af71e>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["SH4"] = df["SH4"].astype(str)


           NO_MUN_MIN SG_UF_MUN_x  \
10013          Jarinu          SP   
15508       Rio Claro          SP   
6502   Gavião Peixoto          SP   
4649            Cotia          SP   
5526          Diadema          SP   

                                              NO_SH4_POR  VALOR_AGREGADO  
10013  Elementos químicos radioactivos e isótopos rad...   129710.000000  
15508  Platina, em formas brutas ou semimanufacturada...   110908.666667  
6502   Binóculos, lunetas, incluídas as astronómicas,...   101287.000000  
4649   Elementos químicos radioactivos e isótopos rad...   100065.000000  
5526   Platina, em formas brutas ou semimanufacturada...    63995.000000  


<ipython-input-5-d765204af71e>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["CO_MUN"] = df["CO_MUN"].astype(str)
<ipython-input-5-d765204af71e>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["SH4"] = df["SH4"].astype(str)


...

      NO_MUN_MIN SG_UF_MUN_x  \
22909  São Roque          SP   
8431    Ibitinga          SP   
22834  São Roque          SP   
12938   Mirassol          SP   
10830     Jarinu          SP   

                                              NO_SH4_POR  VALOR_AGREGADO  
22909  Torneiras, válvulas (incluídas as redutoras de...   200946.000000  
8431   Produtos de origem animal, não especificados n...   190195.000000  
22834  Bússolas, incluídas as agulhas de marear; outr...   153040.526786  
12938  Artefactos de joalharia e suas partes, de meta...   132175.000000  
10830  Elementos químicos radioactivos e isótopos rad...   119360.000000  


<ipython-input-5-d765204af71e>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["CO_MUN"] = df["CO_MUN"].astype(str)
<ipython-input-5-d765204af71e>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["SH4"] = df["SH4"].astype(str)


        NO_MUN_MIN SG_UF_MUN_x  \
15729  Pirapozinho          SP   
1672      Barretos          SP   
23096    São Roque          SP   
466      Andradina          SP   
1683      Barretos          SP   

                                              NO_SH4_POR  VALOR_AGREGADO  
15729  Âmbar-cinzento, castóreo, algália e almíscar; ...   179917.000000  
1672   Produtos de origem animal, não especificados n...   161231.000000  
23096  Quadros, painéis, consolas, cabinas, armários ...   153299.000000  
466    Âmbar-cinzento, castóreo, algália e almíscar; ...   136800.000000  
1683   Âmbar-cinzento, castóreo, algália e almíscar; ...   125492.290905  


<ipython-input-5-d765204af71e>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["CO_MUN"] = df["CO_MUN"].astype(str)
<ipython-input-5-d765204af71e>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["SH4"] = df["SH4"].astype(str)


        NO_MUN_MIN SG_UF_MUN_x  \
16103  Pirapozinho          SP   
23588    São Roque          SP   
22817    São Paulo          SP   
1726      Barretos          SP   
23510    São Roque          SP   

                                              NO_SH4_POR  VALOR_AGREGADO  
16103  Âmbar-cinzento, castóreo, algália e almíscar; ...   226371.750000  
23588  Partes reconhecíveis como exclusiva ou princip...   193207.333333  
22817                                   Energia elétrica   178894.634951  
1726   Âmbar-cinzento, castóreo, algália e almíscar; ...   165547.574187  
23510  Bússolas, incluídas as agulhas de marear; outr...   146504.683333  
